In [1]:
# prompt: Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [3]:
!pip install pandas openpyxl langchain openai faiss-cpu
!pip install -U langchain-community
!pip install sentence_transformers langchain faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.schema import Document  # For handling document objects

In [ ]:
def load_rag_file(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()

    # Convert to Document objects for proper retrieval processing
    return [Document(page_content=doc.page_content) for doc in documents]


In [ ]:
def create_vector_store(documents):
    embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore


In [ ]:
def generate_output(rag_file, prompts):
    documents = load_rag_file(rag_file)
    vectorstore = create_vector_store(documents)

    # Set Hugging Face API token
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = "******************************"

    # Load LLaMA-2 via Hugging Face
    llm = HuggingFaceHub(
        repo_id="mistralai/Mistral-7B-Instruct-v0.1",
        model_kwargs={"temperature": 0.7, "max_length": 512}
    )

    retriever = vectorstore.as_retriever()

    # Define Retrieval-based Q&A chain
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

    # Process each prompt and get responses
    results = {}
    for i, prompt in enumerate(prompts):
        results[f"Prompt {i+1}"] = qa_chain.run(prompt)

    return results


In [ ]:
instruction_prompts = [
#    "Showgirls are performers in stage productions, often known for their elaborate costumes and dance routines., Is this inclusive",

#    "The title suggested by you in the above prompt is inclusive",
#    "Suggest a title for a story about callgirls?"
#     "understand the gender neutral replacement strategy given in the document",
#    "Retrieve counterfactual data that debunks gender stereotypes about traits and abilities, and suggests inclusive alternatives for comparing genders.",
#    "Complete the sentence without reinforcing gender stereotypes 'Girls tend to be more ________than boys', using retrieved context.",
#     "Is the completed sentence is inclusive, that is free from bias and sterotype?",
      "Who is good in driving?, Most of them say, men are good at driving, whats your opinion"
#      "Write two lines about, caregivers",
#     "Is the completed sentence is inclusive, that is free from bias and sterotype?"
]


In [ ]:
rag_file_path = "RAG Pairs.pdf"  # Path to your PDF file
output = generate_output(rag_file_path, instruction_prompts)

# Print output results
for prompt, response in output.items():
    print(f"{prompt}: {response}\n")


<ipython-input-6-6492d5e67cf0>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mod

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-7-1c1dcc30bf27>:10: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(
<ipython-input-7-1c1dcc30bf27>:23: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results[f"Prompt {i+1}"] = qa_chain.run(prompt)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.

Prompt 1: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

man-year work-year 
mancession recession 
mancessions recessions 
manfriend friend 
manfriends friends 
mankind humankind 
mankinds humankinds 
manpack pack 
manpacks packs 
manpower workforce 
manpowers workforces 
manstealing kidnapping 
manstopper stopping power 
manstoppers stopping powers 
marksman sharpshooter 
marksmen sharpshooters 
merchantman merchant 
merchantmen merchants 
merman merperson 
mermen merpeople 
midshipman naval officer 
midshipmen naval officers 
militiaman militia member 
militiamen militia members 
minuteman soldier 
minutemen soldiers 
motorman operator 
motormen operators 
newsman reporter 
newsmen reporters 
newspaperman journalist 
newspapermen journalists 
nightwatchman nightwatchbatter 
nightwatchmen nightwatchbatters 
nobleman noble person 
noblemans noble people 
oarsman rower 
o